# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:

# # Save as a csv
# # Note to avoid any issues later, use encoding="utf-8"
# weather_data.to_csv(output_data_file, encoding="utf-8", index=False)

# Input File (CSV)
input_data_path = "output_data/cities.csv"

# Read the mouse data and the study results
vacation_df = pd.read_csv(input_data_path)


# Display the data table for preview
# print("**************************************************************************")
# print("Reducing datafame size to 8 ([0:8] so need to remove the [0:8] at the end")
# vacation_df=vacation_df_input[0:8]
# print("**************************************************************************")
vacation_df

,city,lat,lon,temp,TempMax,Humidity,Cloudiness,Wind Speed
0,new norfolk,-42.7826,147.0587,19.96,22.64,38,7,1.34
1,mataura,-46.1927,168.8643,23.44,23.44,55,18,2.45
2,peniche,39.3558,-9.3811,11.31,13.68,82,79,3.55
3,port said,31.2565,32.2841,12.93,12.93,67,88,10.04
4,ancud,-41.8697,-73.8203,9.43,9.57,77,86,3.19
...,...,...,...,...,...,...,...,...
575,sosnovo-ozerskoye,52.5249,111.5418,-17.24,-17.24,88,100,4.12
576,kilindoni,-7.9139,39.6668,27.45,27.45,77,100,5.12
577,tanout,14.9709,8.8879,14.49,14.49,11,37,5.98
578,vangaindrano,-23.3500,47.6000,26.65,26.65,83,32,1.83


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
 # Store latitude and longitude in locations
locations = vacation_df[["lat", "lon"]]
# print("locations")
# print(locations)
# print()

# Fill NaN values and convert to float
humidity_weight = (vacation_df["Humidity"].astype(float)) /10
# print("**************************************************************************")
# print("Do I need to get rid of hardcoding that divides by 10 above")

# print("humidity_weight")
# print(humidity_weight)
# print()

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Save the figure
plt.savefig("output_data/lat_lon_hum.png")

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# print('vacation_df')
# print(vacation_df)
# Narrow down criteria to where my family would like to go on vacation (Hint: My wife Diane likes warmer weather:)
#Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 37 Celsius (~98 degrees farneheight) but warmer than 22 (~72 farneheight)
# Wind speed less than or equal to 13kM (~20 mph) 

cities_high_humidity = []
cities_colder_weather = []
cities_high_wind = []

city_counter=0

# print('vacation_df["Humidity"]')
# print(vacation_df["Humidity"])
 
for city in vacation_df:
    if int(vacation_df["Humidity"][city_counter]) > 100:
              # Create a new list to track high humifity index values
              cities_high_humidity.append(city_counter)
    
    if (int(vacation_df["TempMax"][city_counter]) > 37 or int(vacation_df["TempMax"][city_counter]) < 22):
              # Create a new list to track high humifity index values
              cities_colder_weather.append(city_counter)
      
    if int(vacation_df["Wind Speed"][city_counter]) > 13:
              # Create a new list to track high humifity index values
              cities_high_wind.append(city_counter)
    
    city_counter+=1

# Merge/simplify list of unwanted cities based on criteria 
merged_removal_list = cities_high_humidity + cities_high_wind + cities_colder_weather + cities_high_wind
merged_removal_list = list(set(merged_removal_list))
# print("merged_removal_list after remoal of dups and reorder")
# print(merged_removal_list)
# print()
clean_vacation_df = vacation_df
clean_vacation_df=clean_vacation_df.drop(merged_removal_list, inplace=False) 

# drop all rows with any NaN and NaT values
clean_vacation_df = clean_vacation_df.dropna()
# print("clean_vacation_df after rdrop all rows with any NaN and NaT values")
# print(clean_vacation_df)
# print()

clean_vacation_df.reset_index(inplace=True)
print("clean_vacation_df after remoal of unwanted cities and reorder")
clean_vacation_df

clean_vacation_df after remoal of unwanted cities and reorder


,index,city,lat,lon,temp,TempMax,Humidity,Cloudiness,Wind Speed
0,0,new norfolk,-42.7826,147.0587,19.96,22.64,38,7,1.34
1,1,mataura,-46.1927,168.8643,23.44,23.44,55,18,2.45
2,7,mahajanga,-15.7167,46.3167,26.01,26.01,94,40,2.06
3,8,port lincoln,-34.7333,135.8667,21.01,21.01,56,13,6.46
4,9,pacific grove,36.6177,-121.9166,14.58,17.12,51,0,2.06
...,...,...,...,...,...,...,...,...,...
570,575,sosnovo-ozerskoye,52.5249,111.5418,-17.24,-17.24,88,100,4.12
571,576,kilindoni,-7.9139,39.6668,27.45,27.45,77,100,5.12
572,577,tanout,14.9709,8.8879,14.49,14.49,11,37,5.98
573,578,vangaindrano,-23.3500,47.6000,26.65,26.65,83,32,1.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Store into variable named hotel_df
hotel_df = clean_vacation_df
print('**************************************************')
print('Limiting hotel_df to only hotel_df[0:5]')
hotel_df = hotel_df[0:5]
print('**************************************************')
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

**************************************************
Limiting hotel_df to only hotel_df[0:5]
**************************************************


C:\Users\BZNORT~1\AppData\Local\Temp/ipykernel_2756/4186973828.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,index,city,lat,lon,temp,TempMax,Humidity,Cloudiness,Wind Speed,Hotel Name
0,0,new norfolk,-42.7826,147.0587,19.96,22.64,38,7,1.34,
1,1,mataura,-46.1927,168.8643,23.44,23.44,55,18,2.45,
2,7,mahajanga,-15.7167,46.3167,26.01,26.01,94,40,2.06,
3,8,port lincoln,-34.7333,135.8667,21.01,21.01,56,13,6.46,
4,9,pacific grove,36.6177,-121.9166,14.58,17.12,51,0,2.06,


In [37]:
# geocoordinates
target_coordinates = "43.6187102, -116.2146068"
# print('hotel_df[["lat", "lon"]]')
# print(hotel_df[["lat", "lon"]])
# target_coordinates = hotel_df[["lat", "lon"]]
# print('target_coordinates[1:2]')
# print(target_coordinates[1:2])

# target_search = "Chinese"
# target_radius = 8000
# target_type = "restaurant"
target_search = "Marriott"
target_radius = 50000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key  #api_keys  #gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))
print()

# Print the name and address of the first restaurant that appears
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.60528000000001,
                    "lng": -116.2145046
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.60655517989272,
                        "lng": -116.2130459201073
                    },
                    "southwest": {
                        "lat": 43.60385552010727,
                        "lng": -116.2157455798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Residence Inn by Marriott Boise Downtown/University",
            "photos": [
         

In [38]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Residence Inn by Marriott Boise Downtown/University
1401 S Lusk Ave, Boise


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map

# Create a list containing coordinates
# coordinates = [
#     (40.71, -74.00),
#     (30.26, -97.74),
#     (46.87, -96.78),
#     (47.60, -122.33),
#     (32.71, -117.16)
#     ]

print('hotel_df[["lat", "lon"]]')
print(hotel_df[["lat", "lon"]])
coordinates = hotel_df[["lat", "lon"]]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)  # Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

hotel_df[["lat", "lon"]]
       lat       lon
0 -42.7826  147.0587
1 -46.1927  168.8643
2 -15.7167   46.3167
3 -34.7333  135.8667
4  36.6177 -121.9166


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…